In [1]:
from transformers import AutoModel
from transformers import GPT2Tokenizer
from langchain_community.llms import Ollama
from langchain_core.prompts import SystemMessagePromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import PromptTemplate

/Users/wandrilleflamantphospho/Documents/edge-ai-hackathon/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [3]:
def load_llama3() -> Ollama:
    """
    Load the LLAMA3 model from OLLAMA.
    """

    ollama_client = Ollama(base_url="http://localhost:11434", model="llama3")

    system_message = (
        """You are an intelligent scheduling assistant. Your job is to convert short, informal instructions into polite and professional messages that can be sent directly to the recipient to request a service or schedule an appointment. 
    
    Analyze the input to extract key details:
     - Time and date of the meeting or service.
     - Recipient type (e.g., barber, doctor, restaurant, etc.).
     - Service requested (if provided).

    Construct a polite and concise message based on this information. The tone should be courteous and professional.

    Examples:
    Input: 3PM today barber
    Output: Hi, I would like to schedule a haircut appointment today at 3 PM. Is this time available?
    Input: 5 PM tomorrow Italian restaurant for 2
    Output: Hello, I’d like to reserve a table for two at your Italian restaurant tomorrow at 5 PM. Is that possible?
    Input: 9 AM dentist next Tuesday
    Output: Good morning, I’d like to schedule a dental appointment next Tuesday at 9 AM. Does that work for your schedule?

    Guidelines:
    If the input does not specify all details, make a general inquiry.

    For example:
    Input: Barber appointment
    Output: Hello, I’d like to book a haircut appointment. What times are available?
    Always keep the tone polite and professional."""
    )

    SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=[],  # No input variables for the system message
            template=system_message,
        )
    )
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(input_variables=["input"], template="{input}")
    )

    return ollama_client

In [4]:
def generate_answer_ollama(
    input_text: str
) -> str:
    """
    Generate a summary for a given text using the OLLAMA model.
    """

    ollama_model = load_llama3()
    prompt = f"\n\nInput: {input_text}\n\nConstruct a polite and concise message based on this information.\n\nOutput:"

    response = ollama_model(prompt)

    return response

In [5]:
generate_answer_ollama("Between 2AM and 3PM tomorrow, Drink with Nico.")

/var/folders/ht/gnfxwh354lgdzms1j7hx2hp80000gn/T/ipykernel_39793/2672635637.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  ollama_client = Ollama(base_url="http://localhost:11434", model="llama3")
/var/folders/ht/gnfxwh354lgdzms1j7hx2hp80000gn/T/ipykernel_39793/3569868309.py:11: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = ollama_model(prompt)


'"Hey! Would you like to grab a drink sometime between 2am and 3pm tomorrow? I\'d love to catch up with you then. Let me know if that works for you!"'

In [6]:
generate_answer_ollama("lunch italian restaurant 8 people")

'Here\'s a polite and concise message:\n\n"Hi, I\'d like to make a reservation at your Italian restaurant for an upcoming lunch. We\'re a group of 8 people and would like to dine with you on [insert date]. Could you please let me know if you have availability at that time? Thank you!"'

In [10]:
from transformers import pipeline
from datasets import load_dataset
import soundfile as sf
import torch

synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
# You can replace this embedding with your own as well.

speech = synthesiser(generate_answer_ollama("Barber next monday afternon."), forward_params={"speaker_embeddings": speaker_embedding})

sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
